In [15]:
!pip uninstall torch -y
!pip install torch==2.1.0+cu118 -f https://download.pytorch.org/whl/torch_stable.html

^C


Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ---------------------------------------- 0.0/2.7 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.7 GB 41.8 MB/s eta 0:01:06
     ---------------------------------------- 0.0/2.7 GB 40.9 MB/s eta 0:01:07
     ---------------------------------------- 0.0/2.7 GB 41.8 MB/s eta 0:01:06
     ---------------------------------------- 0.0/2.7 GB 42.2 MB/s eta 0:01:05
     ---------------------------------------- 0.0/2.7 GB 42.4 MB/s eta 0:01:05
     ---------------------------------------- 0.0/2.7 GB 43.7 MB/s eta 0:01:03
     ---------------------------------------- 0.0/2.7 GB 36.4 MB/s eta 0:01:15
     ---------------------------------------- 0.0/2.7 GB 36.4 MB/s eta 0:01:15
     ---------------------------------------- 0.0/2.7 GB 38.5 MB/s eta 0:01:11
     ---------------------------------------- 0.0/2.7 GB 38.5 MB/s eta 0:01:11
     ---------------------------------------- 0.0/2.7 GB 46.7 MB/s eta

In [11]:
!pip uninstall torchtext -y
!pip install torchtext==0.16.0


Found existing installation: torchtext 0.18.0
Uninstalling torchtext-0.18.0:
  Successfully uninstalled torchtext-0.18.0
  Using cached torchtext-0.16.0-cp38-cp38-win_amd64.whl.metadata (7.5 kB)
  Using cached torch-2.1.0-cp38-cp38-win_amd64.whl.metadata (24 kB)
Using cached torchtext-0.16.0-cp38-cp38-win_amd64.whl (1.9 MB)
Using cached torch-2.1.0-cp38-cp38-win_amd64.whl (192.3 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.3.0
    Uninstalling torch-2.3.0:
      Successfully uninstalled torch-2.3.0


In [1]:

# use pytorch_env
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd


# Load the dataset
data = pd.read_csv(r'C:\Deep-Learning-based-Authorship-Identification\sun_data.csv')


c:\Users\marti\miniconda3\envs\csci2470project2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
torch.cuda.is_available()

True

In [3]:


# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the data
encodings = tokenizer(data['text'].tolist(), truncation=True, padding='max_length', max_length=128, return_tensors="pt")
input_ids = encodings['input_ids']
attention_mask = encodings['attention_mask']

# Encode the labels
authors = data['author'].unique().tolist()
author_to_id = {author: idx for idx, author in enumerate(authors)}
labels = torch.tensor(data['author'].map(author_to_id).tolist())


In [4]:
# Create the TensorDataset
dataset = TensorDataset(input_ids, attention_mask, labels)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)


In [5]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(authors))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
from torch.optim import AdamW

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)


In [8]:
from tqdm import tqdm

# Training loop
for epoch in range(20):  # number of epochs
    model.train()
    for batch in tqdm(train_loader):
        batch = tuple(item.to(device) for item in batch)
        b_input_ids, b_input_mask, b_labels = batch
        model.zero_grad()
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    
    # Validation phase
    model.eval()
    total_eval_accuracy = 0
    for batch in val_loader:
        batch = tuple(item.to(device) for item in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        total_eval_accuracy += torch.sum(predictions == b_labels).item()

    avg_val_accuracy = total_eval_accuracy / len(val_dataset)
    print(f'Epoch {epoch + 1} Validation Accuracy: {avg_val_accuracy:.4f}')


100%|██████████| 230/230 [00:24<00:00,  9.29it/s]


Epoch 1 Validation Accuracy: 0.7043


100%|██████████| 230/230 [00:23<00:00,  9.61it/s]


Epoch 2 Validation Accuracy: 0.7978


100%|██████████| 230/230 [00:23<00:00,  9.70it/s]


...

Epoch 18 Validation Accuracy: 0.8717


100%|██████████| 230/230 [00:23<00:00,  9.94it/s]


Epoch 19 Validation Accuracy: 0.8696


100%|██████████| 230/230 [00:23<00:00,  9.94it/s]


Epoch 20 Validation Accuracy: 0.8717


In [9]:
# Evaluate on test set
model.eval()
total_eval_accuracy = 0
total_eval_loss = 0

for batch in val_loader:
    batch = tuple(item.to(device) for item in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

    logits = outputs.logits
    loss = outputs.loss
    total_eval_loss += loss.item()
    
    predictions = torch.argmax(logits, dim=-1)
    total_eval_accuracy += torch.sum(predictions == b_labels).item()

avg_test_accuracy = total_eval_accuracy / len(val_loader.dataset)
avg_test_loss = total_eval_loss / len(val_loader.dataset)
print(f"Test Loss: {avg_test_loss:.4f}, Test Accuracy: {avg_test_accuracy:.4f}")


Test Loss: 0.0671, Test Accuracy: 0.8587


In [10]:
def predict_text(text, model, tokenizer):
    # Encode the text
    encoding = tokenizer(text, truncation=True, padding='max_length', max_length=128, return_tensors="pt").to(device)
    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']

    # Make prediction
    model.eval()
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    predictions = torch.argmax(logits, dim=-1)

    # Convert predictions to the label name
    author_id_to_name = {v: k for k, v in author_to_id.items()}
    predicted_author = author_id_to_name[predictions.item()]

    return  probabilities,predicted_author


In [11]:
sample_text = """
The most merciful thing in the world, I think, is the inability of the human mind to correlate all its contents. We live on a placid island of ignorance in the midst of black seas of infinity, and it was not meant that we should voyage far. The sciences, each straining in its own direction, have hitherto harmed us little; but some day the piecing together of dissociated knowledge will open up such terrifying vistas of reality, and of our frightful position therein, that we shall either go mad from the revelation or flee from the deadly light into the peace and safety of a new dark age.
"""
predicted_author, probabilities = predict_text(sample_text, model, tokenizer)
print(f"Predicted Author: {predicted_author}")
print(f"Probabilities: {probabilities}")


Predicted Author: Jules Verne
Probabilities: tensor([[6.3142e-04, 2.9963e-02, 3.5807e-03, 1.7033e-03, 4.2942e-01, 3.9036e-04,
         5.5837e-03, 2.1380e-03, 1.8091e-03, 1.6109e-02, 1.8992e-04, 9.2895e-03,
         2.1972e-02, 2.8554e-02, 1.9690e-03, 2.8256e-03, 4.1582e-01, 1.8547e-04,
         2.1517e-03, 1.0618e-02, 2.9256e-04, 4.9769e-03, 9.8240e-03]],
       device='cuda:0')


In [12]:
import torch

# Assume 'model' is your trained model.
torch.save(model.state_dict(), 'transformer_model.pth')


In [13]:
import os
print(os.getcwd())


c:\Users\marti\Downloads


,text,author_num
0,Happy families are all alike; every unhappy fa...,13
1,"""Ah, ah, ah! Oo!..."" he muttered, recalling ev...",13
2,But he felt all the difficulty of his position...,13
3,"Stepan Arkadyevitch's eyes asked: ""Why do you ...",13
4,"Beg her forgiveness, sir. There's no help for ...",13
...,...,...
13149,"He returned after midnight, and, instead of go...",5
13150,“Strange happiness! If you would hear me witho...,5
13151,"The following evening was very wet: indeed, it...",5
13152,"Hareton, with a streaming face, dug green sods...",5


In [22]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len=512):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=False,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        input_ids = inputs['input_ids'].flatten()
        attention_mask = inputs['attention_mask'].flatten()

        return input_ids, attention_mask

In [24]:
reversed_dict = {
    'Agatha Christie': 0,
    'Arthur Conan Doyle': 1,
    'Austen': 2,
    'Daniel Defoe': 3,
    'Dickens': 4,
    'Emily Bronte': 5,
    'Fitzgerald': 6,
    'Frances Hodgson Burnett': 7,
    'Gaston Leroux': 8,
    'H.P. Lovecraft': 9,
    'Herman Melville': 10,
    'J.K. Rowling': 11,
    'Jules Verne': 12,
    'Leo Tolstoy': 13,
    'Louisa May Alcott': 14,
    'Mark Twain': 15,
    'Mitchell': 16,
    'Montgomery': 17,
    'Robert Louis Stevenson': 18,
    'Shelley': 19,
    'Stoker': 20,
    'Victor Hugo': 21,
    'Wilkie Collins': 22
}

total_dataset = pd.read_csv(r'C:\Deep-Learning-based-Authorship-Identification\data_with_genre.csv')

total_dataset['author_num'] = total_dataset['author'].map(reversed_dict)
total_dataset = total_dataset.drop(columns = ['book_name','author','genre'])
total_dataset


,text,author_num
0,Happy families are all alike; every unhappy fa...,13
1,"""Ah, ah, ah! Oo!..."" he muttered, recalling ev...",13
2,But he felt all the difficulty of his position...,13
3,"Stepan Arkadyevitch's eyes asked: ""Why do you ...",13
4,"Beg her forgiveness, sir. There's no help for ...",13
...,...,...
13149,"He returned after midnight, and, instead of go...",5
13150,“Strange happiness! If you would hear me witho...,5
13151,"The following evening was very wet: indeed, it...",5
13152,"Hareton, with a streaming face, dug green sods...",5


In [25]:

total_dataset = list(total_dataset['text'])
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
total_dataset = TextDataset(total_dataset, tokenizer)
total_loader = DataLoader(total_dataset, batch_size=32, shuffle=False)


In [28]:
import numpy as np
def predict(model, data_loader, device):
    model.eval()
    all_probs = []

    with torch.no_grad():
        for input_ids, attention_mask in data_loader:
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=-1)
            all_probs.append(probs.cpu().numpy())

    # If you need a single matrix for all data
    all_probs = np.vstack(all_probs)  # This stacks all batch predictions into a single array
    return all_probs


In [29]:
all_probs = predict(model,total_loader,device)

In [31]:
all_probs.shape

(13154, 23)

In [32]:
np.savetxt('transformer_prediction.csv', all_probs, delimiter=',')

# For a more feature-rich approach, consider using pandas: